In [233]:
import pandas as pd
import pip
#! pip install apyori
from apyori import apriori
pd.options.display.max_columns = 40
pd.options.display.max_rows = 1000

# Data Manipulation

In [234]:
# Read in the data and fixing the encoding problems
df_with34_55 = pd.read_csv('10000rows.csv',encoding = 'GB2312')
# drop the first columns which is useless
df_with34_55.drop(df_with34_55.columns[0],axis=1,inplace=True)

FileNotFoundError: [Errno 2] File b'10000rows.csv' does not exist: b'10000rows.csv'

In [222]:
# dropping all unused columns
df_with34_55.drop(columns=['IF_CAR_NETWORK',
                 'CNT_TEL',
                  'Role',
                 'dt_Cadillac_CLUB',
                 'Cadillac_point_accu',
                'IF_Buick_CLUB_FLAG',
                'level_Buick',
                'dt_Buick_CLUB',
                'Buick_point_accu',
                'IF_chevy_CLUB_FLAG',
                'level_chevy',
                'dt_chevy_CLUB',
                'chevy_point_accu',
                'CNT_YB_FEE',
                'service_purchase',
                          'IF_CLUB_CAR',
                          'IF_Cadillac_CLUB_FLAG',
                          'contact_CNT_TEL',
                          'DELIVERER_CNT_TEL',
                          'member_CNT_TEL'],axis=1,inplace=True)
df=df_with34_55.drop(columns=['LASTY_CNT_CGBY',
                          'LASTY_AMT_CGBY',
                         'LASTY_CNT_CLDX',
                         'LASTY_AMT_CLDX',
                         'LASTY_CNT_ZBSP',
                         'LASTY_AMT_ZBSP',
                         'LASTY_CNT_NO_ZBSP',
                         'LASTY_AMT_NO_ZBSP',
                         'LASTY_CNT_CGBY_BN',
                         'LASTY_AMT_CGBY_BN',
                         'LASTY_CNT_PTWX_BN',
                         'LASTY_AMT_PTWX_BN',
                             'LASTY_CNT_PTWX_BY',
                             'LASTY_AMT_PTWX_BY',
                             'LASTY_CNT_SGWX',
                             'LASTY_AMT_SGWX',
                             'LASTY_CNT_BUY_YHCP',
                             'LASTY_CNT_WXZ'],axis=1)

In [223]:
# only include cadillac vehicles
cadillac = df[df['HOLD_GRAND']=='凯迪拉克']
cadillac.drop(cadillac.columns[0],axis=1,inplace=True)
# only include individual costumers, exclude all organizational buyers
cadillac = cadillac[cadillac['IF_ORG_CUST']==0]
# drop Car_model since cadillac only has one model in this dataset
cadillac.drop(columns=['Car_model'],inplace=True)
#rename the cols with mis-spells
new_cols = cadillac.columns.values
new_cols[0]="first_buy_mon"

In [224]:
# make sure 一人一车
cust_ID_count={}
for index,row in cadillac.iterrows():
    if row['CUST_ID'] in cust_ID_count:
        cust_ID_count[row['CUST_ID']]+=1
    else:
        cust_ID_count[row['CUST_ID']]=1

more_than_1_list=[]
for key,value in cust_ID_count.items():
    if value>1:
        more_than_1_list.append(key)

for id in  more_than_1_list:
    cadillac=cadillac[cadillac['CUST_ID'] != id]

In [225]:
# make sure all age>0, all first buy month are not null
cadillac=cadillac.loc[cadillac['Age']>0]
cadillac['first_buy_mon']=cadillac['first_buy_mon'].fillna(-1)
cadillac=cadillac.loc[cadillac['first_buy_mon']>0]

In [226]:
# only include vehicles' MSRP with [50000,600000] (reasonable range)
cadillac['LAST_AMT_MSRP']=cadillac['LAST_AMT_MSRP'].fillna(50001)
cadillac=cadillac[(cadillac['LAST_AMT_MSRP']>50000)]
cadillac=cadillac[(cadillac['LAST_AMT_MSRP']<600000)]
# null MSRP are marked as -1
cadillac.loc[cadillac['LAST_AMT_MSRP']==50001,'LAST_AMT_MSRP']=-1

In [227]:
# handling null values

cadillac['Gender']=cadillac['Gender'].fillna('Unknown')
cadillac['Age']=cadillac['Age'].fillna(-1)
cadillac['Province']=cadillac['Province'].fillna('Unknown')
# -1 already exist in city as 5 tier city, -2 represents null
cadillac['City']=cadillac['City'].fillna(-2)
cadillac['level_Cadillac']=cadillac['level_Cadillac'].fillna('非会员')
cadillac['LASTY_MILEAGE']=cadillac['LASTY_MILEAGE'].fillna(-1)
cadillac['LASTY_AMT_BUY_YHCP']=cadillac['LASTY_AMT_BUY_YHCP'].fillna(0)
cadillac['LASTY_AMT_JS']=cadillac['LASTY_AMT_JS'].fillna(0)
cadillac['LASTY_CNT_ACTIVE_CGBY']=cadillac['LASTY_CNT_ACTIVE_CGBY'].fillna(-100)
cadillac['first_buy_mon']=cadillac['first_buy_mon'].fillna(-1)
cadillac['LASTY_CNT_JZ']=cadillac['LASTY_CNT_JZ'].fillna(-1)

In [228]:
# group packaged into 3 categories
cadillac['package_grouped']='Unknown'
cadillac.loc[cadillac['package'].str.contains('技术'),'package_grouped']='低配'
cadillac.loc[cadillac['package'].str.contains('领先'),'package_grouped']='高配'
cadillac.loc[cadillac['package'].str.contains('豪华'),'package_grouped']='高配'
cadillac.loc[cadillac['package'].str.contains('时尚'),'package_grouped']='普配'
cadillac.loc[cadillac['package'].str.contains('精英'),'package_grouped']='普配'
cadillac.loc[cadillac['package'].str.contains('舒适'),'package_grouped']='普配'

In [229]:
# Add a column to display post_sale_Y
cadillac['post_sale_y']=cadillac['LASTY_AMT_BUY_YHCP']+cadillac['LASTY_AMT_JS']

In [230]:
cadillac['LASTY_CNT_JZ'].describe()

count    713.000000
mean       1.809257
std        1.969593
min       -1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max       10.000000
Name: LASTY_CNT_JZ, dtype: float64

# Prepare ready to be passed to Aporiori API data

In [231]:
# first step construct a list of dictionaries, each dictionary represent a row

# vin number skipped, MSRP waited till packages,if_org is already filtered,HOLD_GRAND all the same
# plan to store used one among the 12 services as a list, with key '12_services' 
# if_type_car all the same, insaic_money not mentioned in meeting,province and 
cadillac_dicts = []
for index,row in cadillac.iterrows():
    row_dict = {}
    
    # Temporarily set the post_sale_y threshold as 7000
    if row['post_sale_y']>7000:
        row_dict['post_sale_y']='High Customer Value'
    else:
        row_dict['post_sale_y']='Low Customer Value'
    
    # vehicles bought less than 18 months are considered new since first maintenance should be prior to that
    if row['first_buy_mon']>0 and row['first_buy_mon']<=12:
        row_dict['first_buy_mon']="1 year vehicle"
    # between a year and a half and 4 years is consider middle age 
    elif row['first_buy_mon']>12 and row['first_buy_mon']<=24:
        row_dict['first_buy_mon']="2 year vehicle"
        
    elif row['first_buy_mon']>24 and row['first_buy_mon']<=36:
        row_dict['first_buy_mon']="3 year vehicle"
    elif row['first_buy_mon']>36 and row['first_buy_mon']<=48:
        row_dict['first_buy_mon']="4 year vehicle"
    elif row['first_buy_mon']>48 and row['first_buy_mon']<=60:
        row_dict['first_buy_mon']="5 year vehicle"
    # more than 4 years is considered old since the max in this dataset is 58 and 75% percentile is 37
    elif row['first_buy_mon']>60:
        row_dict['first_buy_mon']="5 older vehicle"
    else:
        row_dict['first_buy_mon']='Unknown'
     
    
    
    # divide to 13 categories(13 keys), one is AJX not installed, the other 12 are 12 AJX serveices
    # if any one of these 13 applies to a vehicle, then it's dict contain the corresponding key value pair
    # where value is the chinses name of the key
    
    # 12 services can all coexist with the others, AJX not installed can only coexist with CKJCBG（车况检测报告）
    # since a few IF_AJX_cust==0 rows has IF_CKJCBG = 1
    services_12 = ['IF_CMYCYJKQ', 'IF_MDDSZXZ', 'IF_CTWZTS',
       'IF_JJJYXZ', 'IF_SSAXJC', 'IF_BDCLDW', 'IF_QCYKLH', 'IF_QYKMTDH',
       'IF_LBJYXZ', 'IF_AQQNBKZZQZ', 'IF_CKJCBG', 'IF_PZZDQZ']
    used=False
    for service in services_12:
        if row[service]==1:
            used=True
    if used == False:
        row_dict['IF_AJX_cust']='AJX not installed'
    
#     if row['IF_AJX_cust']==0:
#         if row['IF_CKJCBG']==0:
#             row_dict['IF_AJX_cust']='AJX not installed'
#         else:
#             row_dict['IF_AJX_cust']='AJX not installed but 车况检测报告'
    
    
    
    
    # handling gender, applied ffill to gender
    if row['Gender']=='男':
        row_dict['Gender'] = 'Male'
    elif row['Gender']=='女':
        row_dict['Gender'] = 'Female'
    else:
        row_dict['Gender'] = 'Unknown'
        
    
    #handling age, applied ffill to age
    # only less than 20% are older than 40
    if row['Age']<=24 and row['Age']>0:
        row_dict['Age'] = 'School Age'
    elif row['Age']>24 and row['Age']<30:
         row_dict['Age'] = 'Age 20s'
    elif row['Age']>=30 and row['Age']<40:
         row_dict['Age'] = 'Age 30s'
    elif row['Age']>=40:
         row_dict['Age'] = 'Age 40s Older'
    else:
         row_dict['Age'] = 'Unknown'
    
    
    #how to group provinces yet to be discussed, ask Daniel iff combine provinces
    d1=['浙江','上海','福建']
    d2=['江苏','安徽']
    d3=['广东','湖南','广西','江西','海南']
    d4=['四川','湖北','云南','贵州','重庆','新疆','甘肃','青海']
    d5=['山东','陕西','辽宁','吉林','黑龙江','内蒙古','山西','宁夏']
    d6=['北京','河南','河北','天津']
    if row['Province'] in d1:
        row_dict['Sales_district']='Sales_district1浙沪闽'
    elif row['Province'] in d2:
        row_dict['Sales_district']='Sales_district2江苏安徽'
    elif row['Province'] in d3:
        row_dict['Sales_district']='Sales_district3粤湘桂赣琼'
    elif row['Province'] in d4:
        row_dict['Sales_district']='Sales_district4云贵川鄂渝疆甘青'
    elif row['Province'] in d5:
        row_dict['Sales_district']='Sales_district5鲁陕辽吉黑蒙晋宁'
    elif row['Province'] in d6:
        row_dict['Sales_district']='Sales_district6河南河北京津'
    else:
         row_dict['Sales_district']='Unknown'
    
    
    # applied ffill to cities
    if row['City'] == 1:
        row_dict['City']= "1 Tier City"
    elif row['City']==2:
        row_dict['City'] = '2 Tier City'
    elif row['City']==3:
        row_dict['City'] = '3 Tier City'
    elif row['City']==4:
        row_dict['City'] = '4 Tier City'
    elif row['City']==0-1:
        row_dict['City'] = '5 Tier City'
    else:
        row_dict['City']='Unknown'
     
    
    # marriage -1 means other or unkown marital status, 1 means married, 2 means unmarried
    # more than 75% are unknown marital stuatus
    if row['Marriage']==1:
        row_dict['Marriage']= 'Married'
    elif row['Marriage']==2:
        row_dict['Marriage']= 'Unmarried'
    else:
        row_dict['Marriage']= 'Unknown'
      
    
    # null in level cadillac means not a member, already replaced null values with '非会员'
    if row['level_Cadillac']== '普卡':
        row_dict['level_Cadillac']='普卡'
    elif row['level_Cadillac']== '银卡':
        row_dict['level_Cadillac']='银卡'
    elif row['level_Cadillac']== '金卡':
        row_dict['level_Cadillac']='金卡'
    elif row['level_Cadillac']== '白金卡':
        row_dict['level_Cadillac']='白金卡'
    else:
        row_dict['level_Cadillac']='非会员'
    
    
    #insaic_cust 1 means client bought insaic insurance, 0 means bought from other sources
    if row['insaic_cust'] == 1:
        row_dict['insaic_cust']='Bought insaic'
    else:
        row_dict['insaic_cust']= 'Didnot buy insaic'
        
        
    # Warranty 0 means did not extend warranty, 1 means did extend warranty
    if row['Warranty']==1:
        row_dict['Warranty']='Extended Warranty'
    else:
        row_dict['Warranty']='Unextended Warranty'
    
    
    # IF_CAR_LOAN 1 means loaned from GM, 0 mean didnot loan from GM but not sure if loaned from others
    if row['IF_CAR_LOAN']==1:
        row_dict['IF_CAR_LOAN']='Loaned'
    else:
        row_dict['IF_CAR_LOAN']='Unloaned'
        
        
    
    #LASTY_MILEAGE
    # replaced null values with -1
    #count      636.000000
    #mean     16331.234543
    #std       9957.809315
    #min       1708.400613
    #25%       9696.742687
    #50%      14198.489512
    #75%      20111.688624
    #max      65905.044118
    #Name: LASTY_MILEAGE, dtype: float64
    if row['LASTY_MILEAGE']== -1:
        row_dict['LASTY_MILEAGE']='Unknown'
    elif row['LASTY_MILEAGE']<= 8000:
        row_dict['LASTY_MILEAGE']='short Last Mileage'
    elif row['LASTY_MILEAGE']> 8000 and row['LASTY_MILEAGE']<=14000:
        row_dict['LASTY_MILEAGE']='medium Last Mileage'
    elif row['LASTY_MILEAGE']> 14000 and row['LASTY_MILEAGE']<=20000:
        row_dict['LASTY_MILEAGE']='long Last Mileage'
    else:
        row_dict['LASTY_MILEAGE']='Very Long Last Mileage'
        
    # LASTY_CNT_ACTIVE_CGBY decribe how proactively client reserved for matainance services last years
    # numeric value = drop in times- text message notified times
    # need to decide if divide into ranges
    if row['LASTY_CNT_ACTIVE_CGBY']==-100:
        row_dict['LASTY_CNT_ACTIVE_CGBY']='Unknown'
    elif row['LASTY_CNT_ACTIVE_CGBY']> 0:
        row_dict['LASTY_CNT_ACTIVE_CGBY']='Active'
    else:
        row_dict['LASTY_CNT_ACTIVE_CGBY']='Not Active'
        
    #IF_TEST_DRIVE 0 means not sure if test drived, 1 and 2 means test_drived
    if row['IF_TEST_DRIVE']==1 or row['IF_TEST_DRIVE']==2:
        row_dict['IF_TEST_DRIVE']='Test Drived'
    else:
        row_dict['IF_TEST_DRIVE']='Unknown'
     
    
    
    # handling package grouped
    if row['package_grouped']=='低配':
        row_dict['package_grouped']='低配'
    if row['package_grouped']=='普配':
        row_dict['package_grouped']='普配'
    else:
        row_dict['package_grouped']='高配'
        
        
    #handling 去年是否进站
    if row['LASTY_CNT_JZ']>0:
        row_dict['LASTY_CNT_JZ']='去年进站过'
    else:
        row_dict['LASTY_CNT_JZ']='去年没进站'
    
    
    #IF_CMYCYJKQ 车门远程应急开启
    if row['IF_CMYCYJKQ']==1:
        row_dict['IF_CMYCYJKQ']='车门远程应急开启'
        
    
    #IF_MDDSZXZ 目的地设置协助
    if row['IF_MDDSZXZ']==1:
        row_dict['IF_MDDSZXZ']='目的地设置协助'
        
    #IF_CTWZTS 车停位置提示
    if row['IF_CTWZTS']==1:
        row_dict['IF_CTWZTS']='车停位置提示'
        
    #IF_JJJYXZ 紧急救援协助
    if row['IF_JJJYXZ']==1:
        row_dict['IF_JJJYXZ']='紧急救援协助'
        
    #IF_SSAXJC 实时按需检测
    if row['IF_SSAXJC']==1:
        row_dict['IF_SSAXJC']='实时按需检测'
        
    #IF_BDCLDW 被盗车辆定位
    if row['IF_BDCLDW']==1:
        row_dict['IF_BDCLDW']='被盗车辆定位'
    
    #IF_QCYKLH 全程音控导航
    if row['IF_QCYKLH']==1:
        row_dict['IF_QCYKLH']='全程音控导航'
        
    #IF_QYKMTDH 全程免提电话
    if row['IF_QYKMTDH']==1:
        row_dict['IF_QYKMTDH']='全程免提电话'
        
    #IF_LBJYXZ 路边救援协助
    if row['IF_LBJYXZ']==1:
        row_dict['IF_LBJYXZ']='路边救援协助'
        
    #IF_AQQNBKZZQZ 安全气囊拨开自助求助
    if row['IF_AQQNBKZZQZ']==1:
        row_dict['IF_AQQNBKZZQZ']='安全气囊拨开自助求助'
        
    #IF_CKJCBG 车况检测报告
    if row['IF_CKJCBG']==1:
        row_dict['IF_CKJCBG']='车况检测报告'
        
    #IF_PZZDQZ 碰撞自动求助
    if row['IF_PZZDQZ']==1:
        row_dict['IF_PZZDQZ']='碰撞自动求助'
    
    cadillac_dicts.append(row_dict)

In [232]:
# convert list of dicts to list of lists ready to passed to Aporiori package
result_list=[]
for dict in cadillac_dicts:
    row=[]
    for key,value in dict.items():
        if value!='Unknown':
            row.append(value)
    result_list.append(row)
result_list

[['Low Customer Value',
  '4 year vehicle',
  'Male',
  'Age 20s',
  'Sales_district6河南河北京津',
  '1 Tier City',
  'Unmarried',
  '普卡',
  'Didnot buy insaic',
  'Unextended Warranty',
  'Unloaned',
  'short Last Mileage',
  'Active',
  '高配',
  '去年进站过',
  '车门远程应急开启',
  '目的地设置协助',
  '车况检测报告'],
 ['High Customer Value',
  '3 year vehicle',
  'Male',
  'Age 20s',
  'Sales_district1浙沪闽',
  '1 Tier City',
  '普卡',
  'Bought insaic',
  'Unextended Warranty',
  'Unloaned',
  'Very Long Last Mileage',
  'Active',
  '高配',
  '去年进站过',
  '车况检测报告'],
 ['High Customer Value',
  '4 year vehicle',
  'Male',
  'Age 40s Older',
  'Sales_district6河南河北京津',
  '1 Tier City',
  'Married',
  '普卡',
  'Didnot buy insaic',
  'Unextended Warranty',
  'Unloaned',
  'medium Last Mileage',
  'Not Active',
  '高配',
  '去年进站过',
  '目的地设置协助',
  '全程音控导航',
  '车况检测报告'],
 ['Low Customer Value',
  '4 year vehicle',
  'Male',
  'Age 20s',
  '普卡',
  'Didnot buy insaic',
  'Unextended Warranty',
  'Unloaned',
  'medium Last Mileage',
 

# Test Algorithm Effectiveness

In [211]:
# change target demographic group's Y value to 50000
test=cadillac
test.loc[(test['Gender']=='女')&(test['City']==1)&(test['Age']>30)&(test['Age']<40),'post_sale_y']=7001


In [212]:
# first step construct a list of dictionaries, each dictionary represent a row

# vin number skipped, MSRP waited till packages,if_org is already filtered,HOLD_GRAND all the same
# plan to store used one among the 12 services as a list, with key '12_services' 
# if_type_car all the same, insaic_money not mentioned in meeting,province and 
test_dicts = []
for index,row in test.iterrows():
    row_dict = {}
    
    # Temporarily set the post_sale_y threshold as 7000
    if row['post_sale_y']>7000:
        row_dict['post_sale_y']='High Customer Value'
    else:
        row_dict['post_sale_y']='Low Customer Value'
    
    # vehicles bought less than 18 months are considered new since first maintenance should be prior to that
    if row['first_buy_mon']>0 and row['first_buy_mon']<=12:
        row_dict['first_buy_mon']="1 year vehicle"
    # between a year and a half and 4 years is consider middle age 
    elif row['first_buy_mon']>12 and row['first_buy_mon']<=24:
        row_dict['first_buy_mon']="2 year vehicle"
        
    elif row['first_buy_mon']>24 and row['first_buy_mon']<=36:
        row_dict['first_buy_mon']="3 year vehicle"
    elif row['first_buy_mon']>36 and row['first_buy_mon']<=48:
        row_dict['first_buy_mon']="4 year vehicle"
    elif row['first_buy_mon']>48 and row['first_buy_mon']<=60:
        row_dict['first_buy_mon']="5 year vehicle"
    # more than 4 years is considered old since the max in this dataset is 58 and 75% percentile is 37
    elif row['first_buy_mon']>60:
        row_dict['first_buy_mon']="5 older vehicle"
    else:
        row_dict['first_buy_mon']='Unknown'
     
    
    
    # divide to 13 categories(13 keys), one is AJX not installed, the other 12 are 12 AJX serveices
    # if any one of these 13 applies to a vehicle, then it's dict contain the corresponding key value pair
    # where value is the chinses name of the key
    
    # 12 services can all coexist with the others, AJX not installed can only coexist with CKJCBG（车况检测报告）
    # since a few IF_AJX_cust==0 rows has IF_CKJCBG = 1
    services_12 = ['IF_CMYCYJKQ', 'IF_MDDSZXZ', 'IF_CTWZTS',
       'IF_JJJYXZ', 'IF_SSAXJC', 'IF_BDCLDW', 'IF_QCYKLH', 'IF_QYKMTDH',
       'IF_LBJYXZ', 'IF_AQQNBKZZQZ', 'IF_CKJCBG', 'IF_PZZDQZ']
    used=False
    for service in services_12:
        if row[service]==1:
            used=True
    if used == False:
        row_dict['IF_AJX_cust']='AJX not installed'
    
#     if row['IF_AJX_cust']==0:
#         if row['IF_CKJCBG']==0:
#             row_dict['IF_AJX_cust']='AJX not installed'
#         else:
#             row_dict['IF_AJX_cust']='AJX not installed but 车况检测报告'
    
    
    
    
    # handling gender, applied ffill to gender
    if row['Gender']=='男':
        row_dict['Gender'] = 'Male'
    elif row['Gender']=='女':
        row_dict['Gender'] = 'Female'
    else:
        row_dict['Gender'] = 'Unknown'
        
    
    #handling age, applied ffill to age
    # only less than 20% are older than 40
    if row['Age']<=24 and row['Age']>0:
        row_dict['Age'] = 'School Age'
    elif row['Age']>24 and row['Age']<30:
         row_dict['Age'] = 'Age 20s'
    elif row['Age']>=30 and row['Age']<40:
         row_dict['Age'] = 'Age 30s'
    elif row['Age']>=40:
         row_dict['Age'] = 'Age 40s Older'
    else:
         row_dict['Age'] = 'Unknown'
    
    
    #how to group provinces yet to be discussed, ask Daniel iff combine provinces
    d1=['浙江','上海','福建']
    d2=['江苏','安徽']
    d3=['广东','湖南','广西','江西','海南']
    d4=['四川','湖北','云南','贵州','重庆','新疆','甘肃','青海']
    d5=['山东','陕西','辽宁','吉林','黑龙江','内蒙古','山西','宁夏']
    d6=['北京','河南','河北','天津']
    if row['Province'] in d1:
        row_dict['Sales_district']='Sales_district1浙沪闽'
    elif row['Province'] in d2:
        row_dict['Sales_district']='Sales_district2江苏安徽'
    elif row['Province'] in d3:
        row_dict['Sales_district']='Sales_district3粤湘桂赣琼'
    elif row['Province'] in d4:
        row_dict['Sales_district']='Sales_district4云贵川鄂渝疆甘青'
    elif row['Province'] in d5:
        row_dict['Sales_district']='Sales_district5鲁陕辽吉黑蒙晋宁'
    elif row['Province'] in d6:
        row_dict['Sales_district']='Sales_district6河南河北京津'
    else:
         row_dict['Sales_district']='Unknown'
    
    
    # applied ffill to cities
    if row['City'] == 1:
        row_dict['City']= "1 Tier City"
    elif row['City']==2:
        row_dict['City'] = '2 Tier City'
    elif row['City']==3:
        row_dict['City'] = '3 Tier City'
    elif row['City']==4:
        row_dict['City'] = '4 Tier City'
    elif row['City']==0-1:
        row_dict['City'] = '5 Tier City'
    else:
        row_dict['City']='Unknown'
     
    
    # marriage -1 means other or unkown marital status, 1 means married, 2 means unmarried
    # more than 75% are unknown marital stuatus
    if row['Marriage']==1:
        row_dict['Marriage']= 'Married'
    elif row['Marriage']==2:
        row_dict['Marriage']= 'Unmarried'
    else:
        row_dict['Marriage']= 'Unknown'
      
    
    # null in level cadillac means not a member, already replaced null values with '非会员'
    if row['level_Cadillac']== '普卡':
        row_dict['level_Cadillac']='普卡'
    elif row['level_Cadillac']== '银卡':
        row_dict['level_Cadillac']='银卡'
    elif row['level_Cadillac']== '金卡':
        row_dict['level_Cadillac']='金卡'
    elif row['level_Cadillac']== '白金卡':
        row_dict['level_Cadillac']='白金卡'
    else:
        row_dict['level_Cadillac']='非会员'
    
    
    #insaic_cust 1 means client bought insaic insurance, 0 means bought from other sources
    if row['insaic_cust'] == 1:
        row_dict['insaic_cust']='Bought insaic'
    else:
        row_dict['insaic_cust']= 'Didnot buy insaic'
        
        
    # Warranty 0 means did not extend warranty, 1 means did extend warranty
    if row['Warranty']==1:
        row_dict['Warranty']='Extended Warranty'
    else:
        row_dict['Warranty']='Unextended Warranty'
    
    
    # IF_CAR_LOAN 1 means loaned from GM, 0 mean didnot loan from GM but not sure if loaned from others
    if row['IF_CAR_LOAN']==1:
        row_dict['IF_CAR_LOAN']='Loaned'
    else:
        row_dict['IF_CAR_LOAN']='Unloaned'
        
        
    
    #LASTY_MILEAGE
    # replaced null values with -1
    #count      636.000000
    #mean     16331.234543
    #std       9957.809315
    #min       1708.400613
    #25%       9696.742687
    #50%      14198.489512
    #75%      20111.688624
    #max      65905.044118
    #Name: LASTY_MILEAGE, dtype: float64
    if row['LASTY_MILEAGE']== -1:
        row_dict['LASTY_MILEAGE']='Unknown'
    elif row['LASTY_MILEAGE']<= 8000:
        row_dict['LASTY_MILEAGE']='short Last Mileage'
    elif row['LASTY_MILEAGE']> 8000 and row['LASTY_MILEAGE']<=14000:
        row_dict['LASTY_MILEAGE']='medium Last Mileage'
    elif row['LASTY_MILEAGE']> 14000 and row['LASTY_MILEAGE']<=20000:
        row_dict['LASTY_MILEAGE']='long Last Mileage'
    else:
        row_dict['LASTY_MILEAGE']='Very Long Last Mileage'
        
    # LASTY_CNT_ACTIVE_CGBY decribe how proactively client reserved for matainance services last years
    # numeric value = drop in times- text message notified times
    # need to decide if divide into ranges
    if row['LASTY_CNT_ACTIVE_CGBY']==-100:
        row_dict['LASTY_CNT_ACTIVE_CGBY']='Unknown'
    elif row['LASTY_CNT_ACTIVE_CGBY']> 0:
        row_dict['LASTY_CNT_ACTIVE_CGBY']='Active'
    else:
        row_dict['LASTY_CNT_ACTIVE_CGBY']='Not Active'
        
    #IF_TEST_DRIVE 0 means not sure if test drived, 1 and 2 means test_drived
    if row['IF_TEST_DRIVE']==1 or row['IF_TEST_DRIVE']==2:
        row_dict['IF_TEST_DRIVE']='Test Drived'
    else:
        row_dict['IF_TEST_DRIVE']='Unknown'
     
    
    
    # handling package grouped
    if row['package_grouped']=='低配':
        row_dict['package_grouped']='低配'
    if row['package_grouped']=='普配':
        row_dict['package_grouped']='普配'
    else:
        row_dict['package_grouped']='高配'
        
        
    #IF_CMYCYJKQ 车门远程应急开启
    if row['IF_CMYCYJKQ']==1:
        row_dict['IF_CMYCYJKQ']='车门远程应急开启'
        
    
    #IF_MDDSZXZ 目的地设置协助
    if row['IF_MDDSZXZ']==1:
        row_dict['IF_MDDSZXZ']='目的地设置协助'
        
    #IF_CTWZTS 车停位置提示
    if row['IF_CTWZTS']==1:
        row_dict['IF_CTWZTS']='车停位置提示'
        
    #IF_JJJYXZ 紧急救援协助
    if row['IF_JJJYXZ']==1:
        row_dict['IF_JJJYXZ']='紧急救援协助'
        
    #IF_SSAXJC 实时按需检测
    if row['IF_SSAXJC']==1:
        row_dict['IF_SSAXJC']='实时按需检测'
        
    #IF_BDCLDW 被盗车辆定位
    if row['IF_BDCLDW']==1:
        row_dict['IF_BDCLDW']='被盗车辆定位'
    
    #IF_QCYKLH 全程音控导航
    if row['IF_QCYKLH']==1:
        row_dict['IF_QCYKLH']='全程音控导航'
        
    #IF_QYKMTDH 全程免提电话
    if row['IF_QYKMTDH']==1:
        row_dict['IF_QYKMTDH']='全程免提电话'
        
    #IF_LBJYXZ 路边救援协助
    if row['IF_LBJYXZ']==1:
        row_dict['IF_LBJYXZ']='路边救援协助'
        
    #IF_AQQNBKZZQZ 安全气囊拨开自助求助
    if row['IF_AQQNBKZZQZ']==1:
        row_dict['IF_AQQNBKZZQZ']='安全气囊拨开自助求助'
        
    #IF_CKJCBG 车况检测报告
    if row['IF_CKJCBG']==1:
        row_dict['IF_CKJCBG']='车况检测报告'
        
    #IF_PZZDQZ 碰撞自动求助
    if row['IF_PZZDQZ']==1:
        row_dict['IF_PZZDQZ']='碰撞自动求助'
    
    test_dicts.append(row_dict)

In [213]:
test_result_list=[]
for dict in test_dicts:
    row=[]
    for key,value in dict.items():
        if value!='Unknown':
            row.append(value)
    test_result_list.append(row)
test_result_list

[['Low Customer Value',
  '4 year vehicle',
  'Male',
  'Age 20s',
  'Sales_district6河南河北京津',
  '1 Tier City',
  'Unmarried',
  '普卡',
  'Didnot buy insaic',
  'Unextended Warranty',
  'Unloaned',
  'short Last Mileage',
  'Active',
  '高配',
  '车门远程应急开启',
  '目的地设置协助',
  '车况检测报告'],
 ['High Customer Value',
  '3 year vehicle',
  'Male',
  'Age 20s',
  'Sales_district1浙沪闽',
  '1 Tier City',
  '普卡',
  'Bought insaic',
  'Unextended Warranty',
  'Unloaned',
  'Very Long Last Mileage',
  'Active',
  '高配',
  '车况检测报告'],
 ['High Customer Value',
  '4 year vehicle',
  'Male',
  'Age 40s Older',
  'Sales_district6河南河北京津',
  '1 Tier City',
  'Married',
  '普卡',
  'Didnot buy insaic',
  'Unextended Warranty',
  'Unloaned',
  'medium Last Mileage',
  'Not Active',
  '高配',
  '目的地设置协助',
  '全程音控导航',
  '车况检测报告'],
 ['Low Customer Value',
  '4 year vehicle',
  'Male',
  'Age 20s',
  '普卡',
  'Didnot buy insaic',
  'Unextended Warranty',
  'Unloaned',
  'medium Last Mileage',
  'Active',
  '高配',
  '全程音控导航',
  

In [214]:
association_rules = apriori(test_result_list,min_support=0.045,min_confidence=0.2,min_lift=3.5,min_length=2)
for item in association_rules:
    pair = item[0]
    items = [x for x in pair]
    if items[1]=='Low Customer Value' or 'High Customer Value':
        print(item)
        # first index of the inner list
        # Contains base item and add item
        pair = item[0]
        print('pair is:')
        print(pair)
        items = [x for x in pair]
        print("Rule: " + items[0] + " -> " + items[1])

        #second index of the inner list
        print("Support: " + str(item[1]))

        #third index of the list located at 0th
        #of the third index of the inner list

        print("Confidence: " + str(item[2][0][2]))
        print("Lift: " + str(item[2][0][3]))
        print("=====================================")

RelationRecord(items=frozenset({'Married', '5 year vehicle', 'Age 30s'}), support=0.04628330995792426, ordered_statistics=[OrderedStatistic(items_base=frozenset({'5 year vehicle', 'Age 30s'}), items_add=frozenset({'Married'}), confidence=0.6470588235294118, lift=3.5763793889648885), OrderedStatistic(items_base=frozenset({'Married', 'Age 30s'}), items_add=frozenset({'5 year vehicle'}), confidence=0.515625, lift=3.953125)])
pair is:
frozenset({'Married', '5 year vehicle', 'Age 30s'})
Rule: Married -> 5 year vehicle
Support: 0.04628330995792426
Confidence: 0.6470588235294118
Lift: 3.5763793889648885
RelationRecord(items=frozenset({'Didnot buy insaic', '5 year vehicle', 'Married'}), support=0.06732117812061711, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Married', 'Didnot buy insaic'}), items_add=frozenset({'5 year vehicle'}), confidence=0.48484848484848486, lift=3.7171717171717176)])
pair is:
frozenset({'Didnot buy insaic', '5 year vehicle', 'Married'})
Rule: Didnot buy in

RelationRecord(items=frozenset({'Age 30s', 'Female', 'Unextended Warranty', 'High Customer Value', '1 Tier City'}), support=0.04628330995792426, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Unextended Warranty', '1 Tier City', 'Age 30s', 'Female'}), items_add=frozenset({'High Customer Value'}), confidence=0.868421052631579, lift=4.32995951417004)])
pair is:
frozenset({'Age 30s', 'Female', 'Unextended Warranty', 'High Customer Value', '1 Tier City'})
Rule: Age 30s -> Female
Support: 0.04628330995792426
Confidence: 0.868421052631579
Lift: 4.32995951417004
RelationRecord(items=frozenset({'Age 30s', 'Female', 'High Customer Value', '车况检测报告', '1 Tier City'}), support=0.04908835904628331, ordered_statistics=[OrderedStatistic(items_base=frozenset({'车况检测报告', '1 Tier City', 'Age 30s', 'Female'}), items_add=frozenset({'High Customer Value'}), confidence=0.8536585365853658, lift=4.256353402694866)])
pair is:
frozenset({'Age 30s', 'Female', 'High Customer Value', '车况检测报告', '1 Tier C

RelationRecord(items=frozenset({'Age 30s', 'Female', 'Unextended Warranty', 'High Customer Value', '车况检测报告', '1 Tier City'}), support=0.04628330995792426, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Age 30s', 'Female', 'Unextended Warranty', '车况检测报告', '1 Tier City'}), items_add=frozenset({'High Customer Value'}), confidence=0.868421052631579, lift=4.32995951417004)])
pair is:
frozenset({'Age 30s', 'Female', 'Unextended Warranty', 'High Customer Value', '车况检测报告', '1 Tier City'})
Rule: Age 30s -> Female
Support: 0.04628330995792426
Confidence: 0.868421052631579
Lift: 4.32995951417004
RelationRecord(items=frozenset({'5 year vehicle', 'Unextended Warranty', 'Married', 'Didnot buy insaic', 'Low Customer Value', '普配'}), support=0.04908835904628331, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Unextended Warranty', 'Low Customer Value', 'Didnot buy insaic', 'Married', '普配'}), items_add=frozenset({'5 year vehicle'}), confidence=0.5645161290322581, lift=4.32795698